## 1. Thông tin cơ bản về báo cáo
- **VAERS_ID**: Mã số duy nhất để định danh báo cáo, liên kết giữa các file dữ liệu.
- **RECVDATE**: Ngày hệ thống VAERS nhận được báo cáo.
- **STATE**: Bang (Mỹ) của người được tiêm vaccine (viết tắt 2 chữ cái).
- **FORM_VERS**: Phiên bản form VAERS (1 hoặc 2).

## 2. Thông tin nhân khẩu học
- **AGE_YRS**: Tuổi của người được tiêm (tính theo năm).
- **CAGE_YR**: Tuổi tính bằng năm (dựa trên ngày tiêm – ngày sinh).
- **CAGE_MO**: Tuổi tính bằng tháng (bổ sung cho CAGE_YR, ví dụ: CAGE_YR = 1 + CAGE_MO = 0.5 → 1.5 tuổi).
- **SEX**: Giới tính (M: Nam, F: Nữ, trống: Không rõ).

## 3. Thông tin về triệu chứng và kết quả
- **SYMPTOM_TEXT**: Mô tả triệu chứng bằng văn bản (từ người báo cáo).
- **SYMPTOM1–SYMPTOM5**: Các triệu chứng được mã hóa theo từ điển MedDRA (tối đa 5 triệu chứng/báo cáo).
- **SYMPTOMVERSION1–5**: Phiên bản MedDRA dùng để mã hóa triệu chứng.
- **DIED**: Có tử vong không (Y/N).
- **DATEDIED**: Ngày tử vong (nếu có).
- **L_THREAT**: Sự kiện đe dọa tính mạng (Y/N).
- **HOSPITAL**: Nhập viện (Y/N).
- **HOSPDAYS**: Số ngày nhập viện.
- **DISABLE**: Tàn tật sau tiêm (Y/N).
- **RECOVD**: Đã hồi phục (Y/N/U – U: Không rõ).
- **BIRTH_DEFECT**: Dị tật bẩm sinh (chỉ form VAERS 2).

## 4. Thông tin về vaccine
- **VAX_TYPE**: Loại vaccine (ví dụ: COVID19, FLU3 – mã hóa theo bảng trong tài liệu).
- **VAX_MANU**: Nhà sản xuất vaccine.
- **VAX_LOT**: Số lô vaccine.
- **VAX_DOSE_SERIES**: Số mũi tiêm đã nhận.
- **VAX_ROUTE**: Đường tiêm (IM: Tiêm bắp, SC: Dưới da, IN: Qua mũi…).
- **VAX_SITE**: Vị trí tiêm (ví dụ: Cánh tay trái).
- **VAX_NAME**: Tên thương hiệu vaccine.

## 5. Thời gian và sự kiện
- **VAX_DATE**: Ngày tiêm vaccine.
- **ONSET_DATE**: Ngày xuất hiện triệu chứng.
- **NUMDAYS**: Số ngày từ lúc tiêm đến khi có triệu chứng (ONSET_DATE – VAX_DATE).
- **RPT_DATE**: Ngày hoàn thành form báo cáo (chỉ form VAERS 1).
- **TODAYS_DATE**: Ngày hoàn thành form (chỉ form VAERS 2).

## 6. Thông tin y tế khác
- **LAB_DATA**: Kết quả xét nghiệm liên quan (văn bản tự do).
- **OTHER_MEDS**: Thuốc đang dùng khi tiêm.
- **CUR_ILL**: Bệnh lý mắc phải tại thời điểm tiêm.
- **HISTORY**: Tiền sử bệnh mãn tính.
- **ALLERGIES**: Dị ứng (chỉ form VAERS 2).
- **PRIOR_VAX**: Thông tin về vaccine tiêm trước đó.

## 7. Địa điểm và nguồn gốc vaccine
- **V_ADMINBY**: Nơi tiêm (PUB: Công cộng, PVT: Tư nhân, MIL: Quân đội…).
- **V_FUNDBY**: Nguồn chi trả vaccine (chỉ form VAERS 1).

## 8. Các trường khác
- **SPLTTYPE**: Mã nhà sản xuất hoặc dự án tiêm chủng.
- **ER_VISIT**: Khám cấp cứu (chỉ form VAERS 1).
- **ER_ED_VISIT**: Khám cấp cứu/khẩn cấp (chỉ form VAERS 2).
- **OFC_VISIT**: Khám tại phòng khám (chỉ form VAERS 2).


In [ ]:
# Title: CS229 project (converted to Python)
# Author: Xiaojuan, adapted by ChatGPT
# Date: 2025-04-17

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Setup
data_dir = "..\\data\\"
pd.set_option('mode.chained_assignment', None)

# Load datasets
dt1_21 = pd.read_csv(data_dir + "2021VAERSData/2021VAERSDATA.csv", encoding='ISO-8859-1')
dt2_21 = pd.read_csv(data_dir + "2021VAERSData/2021VAERSVAX.csv", encoding='ISO-8859-1')
dt3_21 = pd.read_csv(data_dir + "2021VAERSData/2021VAERSSYMPTOMS.csv", encoding='ISO-8859-1')

dt1_20 = pd.read_csv(data_dir + "2020VAERSData/2020VAERSDATA.csv", encoding='ISO-8859-1')
dt2_20 = pd.read_csv(data_dir + "2020VAERSData/2020VAERSVAX.csv", encoding='ISO-8859-1')
dt3_20 = pd.read_csv(data_dir + "2020VAERSData/2020VAERSSYMPTOMS.csv", encoding='ISO-8859-1')

# Merge by VAERS_ID and filter for COVID19 vaccine
df21 = dt2_21[dt2_21.VAX_TYPE == "COVID19"].merge(dt1_21, on="VAERS_ID").merge(dt3_21, on="VAERS_ID")
df20 = dt2_20[dt2_20.VAX_TYPE == "COVID19"].merge(dt1_20, on="VAERS_ID").merge(dt3_20, on="VAERS_ID")
df = pd.concat([df21, df20], ignore_index=True)



    

# Preprocessing
df = df.copy()
df['othermeds'] = df['OTHER_MEDS'].apply(lambda x: 0 if pd.isna(x) or str(x).strip().lower() in ["", "na"] else 1)
df['curr_ill'] = df['CUR_ILL'].apply(lambda x: 0 if pd.isna(x) or str(x).strip().lower() in ["", "na"] else 1)
df['allergies'] = df['ALLERGIES'].apply(lambda x: 0 if pd.isna(x) or str(x).strip().lower() in ["", "na"] else 1)
df['sex'] = df['SEX'].apply(lambda x: 1 if x == 'F' else 0)
df['disable'] = df['DISABLE'].apply(lambda x: 1 if pd.notna(x) and str(x).strip() != "" else 0)

# Manufacturer encoding
df['manu'] = df['VAX_MANU'].map({'MODERNA': 0, 'PFIZER\\BIONTECH': 2}).fillna(3).astype(int)

# Date processing
df['ONSET_DATE'] = pd.to_datetime(df['ONSET_DATE'], errors='coerce')
df['VAX_DATE'] = pd.to_datetime(df['VAX_DATE'], errors='coerce')

df['ONSET_DATE'] = df['ONSET_DATE'].fillna(method='ffill')
df['VAX_DATE'] = df['VAX_DATE'].fillna(method='ffill')

df = df[(df['ONSET_DATE'].dt.year >= 2020) & (df['VAX_DATE'].dt.year >= 2020)]

df['dur'] = (df['ONSET_DATE'] - df['VAX_DATE']).dt.days
df = df[df['NUMDAYS'] <= 50]

# Histogram of top 15 symptoms
symptoms = pd.concat([df[f'SYMPTOM{i}'] for i in range(1, 6)])
symptoms = symptoms[~symptoms.str.contains("Injection site", na=False)]
hist = symptoms.value_counts().reset_index()
hist.columns = ['symptom', 'Freq']

plt.figure(figsize=(10,6))
sns.barplot(x='symptom', y='Freq', data=hist.head(15), palette='Reds_r')
plt.xticks(rotation=45, ha='right')
plt.xlabel("Symptoms")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

# Daily adverse events over time
daily = df.groupby(df['ONSET_DATE'].dt.date).size().reset_index(name='count')
daily = daily[daily.index >= 55]

plt.figure(figsize=(10,5))
sns.lineplot(x='ONSET_DATE', y='count', data=daily, color='red')
plt.xlabel("Date")
plt.ylabel("Adverse effect events (n)")
plt.tight_layout()
plt.show()

# Daily events by manufacturer
daily_manu = df[df['VAX_MANU'] != "UNKNOWN MANUFACTURER"].groupby(
    [df['ONSET_DATE'].dt.date, 'VAX_MANU']).size().reset_index(name='count')

plt.figure(figsize=(12,6))
sns.lineplot(data=daily_manu[daily_manu.index >= 96], x='ONSET_DATE', y='count', hue='VAX_MANU')
plt.xlabel("Date")
plt.ylabel("Adverse effect events (n)")
plt.tight_layout()
plt.show()


# Baseline feature extraction from HISTORY
def grep_history(pattern, col):
    return col.str.contains(pattern, case=False, na=False).astype(int)

df['Allergic_his'] = grep_history("allergic", df['HISTORY'])
df['diabetes'] = grep_history("diabete", df['HISTORY'])
df['hypertension'] = grep_history("hypertension|high blood", df['HISTORY'])
df['arthritis'] = grep_history("arthritis", df['HISTORY'])
df['Asthma'] = grep_history("asthma", df['HISTORY'])
df['Migraine'] = grep_history("migraine", df['HISTORY'])
df['copd'] = grep_history("copd", df['HISTORY'])
df['Anxiety'] = grep_history("anxiety", df['HISTORY'])
df['obesity'] = grep_history("obesity", df['HISTORY'])
df['depression'] = grep_history("depression", df['HISTORY'])
df['Thyroid'] = grep_history("thyroid", df['HISTORY'])
df['Anemia'] = grep_history("anemia", df['HISTORY'])
df['Dementia'] = grep_history("dementia", df['HISTORY'])
df['Cancer'] = grep_history("cancer", df['HISTORY'])
df['Kidney'] = grep_history("kidney|ckd", df['HISTORY'])
df['Hyperlipidemia'] = grep_history("hyperlipidemia", df['HISTORY'])
df['CVD'] = grep_history("heart|cvd|stroke|hf", df['HISTORY'])
df['AF'] = grep_history("atrial fibrillation|af", df['HISTORY'])

# Final clean & export
df0 = df[np.isfinite(df['VAERS_ID'])]
df0.to_csv(data_dir + "vaers1.csv", index=False, encoding="utf-8")


In [ ]:
import pandas as pd

# 1. Đọc dữ liệu
df0 = pd.read_csv("vaers1.csv")

# 2. Chuyển cột VAX_DATE sang định dạng datetime
df0['VAX_DATE'] = pd.to_datetime(df0['VAX_DATE'], errors='coerce')

# 3. Lọc dữ liệu theo khoảng thời gian từ 20/12/2020 đến 22/4/2021
start_date = pd.to_datetime("2020-12-20")
end_date = pd.to_datetime("2021-04-22")
df_filtered = df0[(df0['VAX_DATE'] >= start_date) & (df0['VAX_DATE'] <= end_date)]

# 4. Lưu dữ liệu đã lọc vào file mới
df_filtered.to_csv("vaers1fill.csv", index=False)

# Kiểm tra một số dòng đầu của dữ liệu đã lọc
print(df_filtered.head())